## Autoencoder 

In [21]:
import torch 
import torch.nn as nn

class F(nn.Module):
    def __init__(self,phi_in:int,phi): #:nn.Module
        """Creates a new instance of F.
        Parameters
        - - - - - -
        phi_in : int
        The input dimension of phi, i.e., the expected
        shape is [batch_size, phi_in]
        phi : nn.Module
        PyTorch module which implements a neural network block "Phi"
        """
        super().__init__()
        self.phi = phi
        self.phi_in = phi_in
        self.xb_size = phi_in[1]
        
    def forward(self, inputs):
        xa,xb=inputs.split([inputs.shape[1]-self.xb_size,self.xb_size],dim=1)
        ya = xa + self.phi(xb)
        yb = xb
        return torch.cat((ya,yb),axis=1)

    def inverse(self, inputs):
        ya,yb=inputs.split([inputs.shape[1]-self.xb_size,self.xb_size],dim=1)
        xb=yb
        xa=ya-self.phi(yb)
        return torch.cat((xa,xb),axis=1)


In [22]:
import unittest

def test_phi(x):
    return x

class TestF(unittest.TestCase):
    def setUp(self):
        phi_in = (3,2) # batch_size = 3
        self.model=F(phi_in,test_phi)
        xa = torch.tensor(((1,2),(1,2),(1,2)))
        xb = torch.tensor(((3,4),(3,4),(3,4)))
        self.x = torch.cat((xa,xb),axis=1)
        ya = torch.tensor(((4,6),(4,6),(4,6)))
        yb = torch.tensor(((3,4),(3,4),(3,4)))
        self.y= torch.cat((ya,yb),axis=1)
        
    def test_forward(self):
        y_pred = self.model(self.x)
        self.assertTrue(torch.equal(y_pred,self.y))
        
    def test_inverse(self):
        x_pred = self.model.inverse(self.y)
        self.assertTrue(torch.equal(x_pred,self.x))
            
unittest.main(argv=[''], verbosity=2, exit=False)

test_forward (__main__.TestF) ... ok
test_inverse (__main__.TestF) ... ok

----------------------------------------------------------------------
Ran 2 tests in 0.032s

OK


In [5]:
import torch
x=torch.tensor(((2,3),(4,5),(6,7)))
x.shape

torch.Size([3, 2])

## Inverse

In [3]:
import torch
import torch.nn as nn

class S(nn.Module):
    def __init__(self, n_A, n_B):
        """ Creates a new instance of S.
        Parameters
        - - - - - -
        n_A : int
        Number of components for x_A
        n_B : int
        Number of components for x_B
        """
        
        super().__init__()
        self.n_A=n_A
        self.n_B=n_B
    
    def forward(self, inputs):
        xa,xb=inputs.split([self.n_A,self.n_B])
        return torch.cat((xb,xa))
    
    def inverse(self, inputs):
        xb,xa=inputs.split([self.n_B,self.n_A])
        return torch.cat((xa,xb))


In [4]:
import unittest

class TestS(unittest.TestCase):
    def setUp(self):
        self.model=S(2,3)
        xa = torch.tensor((1,2))
        xb = torch.tensor((3,4,5))
        self.forward=torch.cat((xa,xb))
        self.backward=torch.cat((xb,xa))
        
    def test_forward(self):
        y_pred = self.model(self.forward)
        self.assertTrue(torch.equal(y_pred,self.backward))
        
    def test_inverse(self):
        y_pred = self.model.inverse(self.backward)
        self.assertTrue(torch.equal(y_pred,self.forward))
                    
unittest.main(argv=[''], verbosity=2, exit=False)

test_forward (__main__.TestF) ... ok
test_inverse (__main__.TestF) ... ok
test_forward (__main__.TestS) ... ok
test_inverse (__main__.TestS) ... ok

----------------------------------------------------------------------
Ran 4 tests in 0.014s

OK
